In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime
from sklearn.cross_validation import train_test_split
import operator

0it [00:00, ?it/s]
/Library/Python/2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [115]:
train = pd.read_csv('./train_1.csv').fillna(0)

In [7]:
key = pd.read_csv('./key_trunc.csv')

In [8]:
sub = pd.read_csv("./sample_submission_1.csv")

In [76]:
train_melt = pd.read_csv("train_feats.csv")

In [234]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

KeyboardInterrupt: 

In [125]:
def get_median(row, last_n_days):
    return pd.rolling_median(pd.Series(row), window = last_n_days, min_periods=7)[-60:]    

In [126]:
def get_mean(row, last_n_days):
    return pd.rolling_mean(pd.Series(row), window = last_n_days, min_periods=7)[-60:]    

In [86]:
def get_mean_diff(row, last_n_days):
    return pd.Series(pd.rolling_mean(np.diff(pd.Series(row)), window = last_n_days))

In [87]:
def get_last_day_visits(row):
    return row[1:]

In [128]:
def get_long_stats(func_type, df, df_melt, last_n_days = None):

    if func_type == "median":
        rolling_stats = df.progress_apply(get_median, last_n_days = last_n_days, axis = 1)
        
    elif func_type == "mean":
        rolling_stats = df.progress_apply(get_mean, last_n_days = last_n_days, axis = 1)
        
#     elif func_type == "mean_diff":
#         rolling_stats = df.iloc[:, -61:].progress_apply(get_mean_diff, last_n_days = last_n_days, axis = 1)
        
#     elif func_type == "last_day_visits":        
#         rolling_stats = df.iloc[:, -61:].progress_apply(get_last_day_visits, axis = 1)

    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)  
    rolling_stats_df.columns = ["Page"] + list(df.columns[-60:])
    
    rolling_stats_df = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))        
    
    df_melt = pd.merge(df_melt, rolling_stats_df, on = ["Page", "date"])     
    
    return df_melt

In [ ]:
def generate_train_melt(train):
    
#     train_melt = pd.read_csv("train_feats.csv")
    
#     Commented because these features are already generated
    train_melt = pd.melt(pd.concat([train.Page, train.iloc[:, -60:]], axis = 1), id_vars=['Page'], var_name="date", value_name="visits")

#     train_melt = get_long_stats("mean", train, train_melt, 7)
    train_melt = get_long_stats("mean", train, train_melt, 30)
    train_melt = get_long_stats("mean", train, train_melt, 60)

#     train_melt = get_long_stats("median", train, train_melt, 7)
    train_melt = get_long_stats("median", train, train_melt, 30)
    train_melt = get_long_stats("median", train, train_melt, 60)

#     train_melt = get_long_stats("mean_diff", train, train_melt, 7)
#     train_melt = get_long_stats("last_day_visits", train, train_melt)
    
    train_melt["date_num"] = train_melt.progress_apply(get_date, axis = 1)
    train_melt["weekday"] = train_melt.progress_apply(get_weekday, axis = 1)
    
    # Columns i am not using
    # train_melt["month"] = train_melt.progress_apply(get_month, axis = 1)
    # train_melt["language"] = train_melt.progress_apply(get_language, axis = 1)
    
    return train_melt 

train_melt = generate_train_melt(train)
train_melt.to_csv("train_feats.csv", index = False)





  0%|          | 0/145063 [00:00<?, ?it/s]



  0%|          | 1/145063 [00:11<472:59:02, 11.74s/it]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=7,window=30,center=False).mean()
  




  0%|          | 2/145063 [00:18<418:29:05, 10.39s/it]



  0%|          | 126/145063 [00:19<292:41:55,  7.27s/it]



  0%|          | 264/145063 [00:19<204:42:10,  5.09s/it]



  0%|          | 405/145063 [00:19<143:09:39,  3.56s/it]



  0%|          | 551/145063 [00:19<100:07:11,  2.49s/it]



  0%|          | 708/145063 [00:19<70:00:55,  1.75s/it] 



  1%|          | 854/145063 [00:19<48:58:10,  1.22s/it]



  1%|          | 1012/145063 [00:19<34:14:55,  1.17it/s]



  1%|          | 1173/145063 [00:19<23:57:17,  1.67it/s]



  1%|          | 1325/145063 [00:19<16:45:30,  2.38it/s]



  1%|          | 1475/145063 [00:19<11:43:36,  3.40it/s]



 

 11%|█▏        | 16470/145063 [00:32<01:43, 1242.12it/s]



 11%|█▏        | 16627/145063 [00:32<01:36, 1324.34it/s]



 12%|█▏        | 16784/145063 [00:32<01:33, 1378.35it/s]



 12%|█▏        | 16940/145063 [00:32<01:30, 1409.14it/s]



 12%|█▏        | 17094/145063 [00:32<01:29, 1435.81it/s]



 12%|█▏        | 17247/145063 [00:32<01:27, 1461.56it/s]



 12%|█▏        | 17400/145063 [00:32<01:26, 1473.15it/s]



 12%|█▏        | 17553/145063 [00:32<01:25, 1489.68it/s]



 12%|█▏        | 17709/145063 [00:33<01:24, 1509.86it/s]



 12%|█▏        | 17873/145063 [00:33<01:22, 1545.55it/s]



 12%|█▏        | 18030/145063 [00:33<01:23, 1516.39it/s]



 13%|█▎        | 18184/145063 [00:33<01:24, 1493.96it/s]



 13%|█▎        | 18335/145063 [00:33<01:25, 1489.21it/s]



 13%|█▎        | 18496/145063 [00:33<01:23, 1521.37it/s]



 13%|█▎        | 18652/145063 [00:33<01:22, 1531.63it/s]



 13%|█▎        | 18809/145063 [00:33<01:21, 1540.97it/s]



 13%|█▎        | 18968/145063 [00:33<01:

In [7]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [8]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [94]:
def get_month(row):
    return row.date.split("-")[1]

In [95]:
def get_date(row):
    return row.date.split("-")[2]

In [96]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [13]:
# def get_language_mean(row):
# lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
# print lang_count

In [22]:
def get_median_test(row, last_index, last_n_days):
    
    start_loc = max(min(last_index - last_n_days, 61), 0)
    end_loc = min(last_index, 61)

    if start_loc < end_loc:
        return np.median(row.iloc[ start_loc : end_loc].tolist())
    else:
        return np.median(row.iloc[61 - last_n_days : 61].tolist())


In [23]:
def get_mean_test(row, last_index, last_n_days):
    
    start_loc = max(min(last_index - last_n_days, 61), 0)
    end_loc = min(last_index, 61)
    
    if start_loc < end_loc:
        return np.mean(row.iloc[start_loc : end_loc].tolist())
    else:
        return np.mean(row.iloc[61 - last_n_days : 61].tolist())

In [12]:
def get_mean_diff_test(row, last_index):
    
    start_loc = min(last_index - 7, 61)
    end_loc = min(last_index, 61)
    
    if start_loc < end_loc:
        return np.mean([int(x) for x in np.diff( row.iloc[start_loc : end_loc] ) ], dtype=None)
    else:
        return np.nan

In [13]:
def get_last_day_visits_test(row, last_index):
    
    return row.iloc[last_index - 1]

In [18]:
# def get_test_feats(row, last_index):
    
#     subset_row = row.iloc[last_index - 60 : last_index].tolist()
    
#     median_7 = np.median(subset_row[-7:])
#     median_30 = np.median(subset_row[-30:])
#     median_60 = np.median(subset_row[-60:])

#     mean_7 = np.mean(subset_row[-7:])
#     mean_30 = np.mean(subset_row[-30:])
#     mean_60 = np.mean(subset_row[-60:])
    
#     return pd.Series([row.Page, median_7, median_30, median_60, mean_7, mean_30, mean_60])

In [14]:
def smape(y_pred, y_true):
        
    y_true = y_true.get_label()
    y_pred = np.array(y_pred)
        
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0

    return "smape", np.nanmean(diff)

In [38]:
def run_xgb(train, label_train, valid = None, label_valid = None):

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'reg:linear'
    params['eta'] = 0.05
    params['max_depth'] = 7
    params['silent'] = 1
    params['min_child_weight'] = 0
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['nthread'] = 13
    params['gamma'] = 0
    params['max_delta_step'] = 0

    d_train = xgb.DMatrix(train, label=label_train)
    
    if valid is not None:
        d_valid = xgb.DMatrix(valid, label=label_valid)
        watchlist = [(d_train, 'train'), (d_valid, 'validation')]
    else:
        watchlist = [(d_train, 'train')]
        
    bst = xgb.train(params, d_train, 100, watchlist, early_stopping_rounds=10, verbose_eval=10, feval=smape, maximize = False)
    
    return bst

In [79]:
label_train_melt = train_melt["visits"]
# label_train_melt = [0] + np.diff(train_melt["visits"]).tolist()

train_melt.drop("visits", axis = 1, inplace = True)
train_melt.drop("date", axis = 1, inplace = True)
train_melt.drop("Page", axis = 1, inplace = True)

train_melt.drop("last_day_visits_None", axis = 1, inplace = True)
train_melt.drop("mean_diff_7", axis = 1, inplace = True)

train_melt.drop("mean_7", axis = 1, inplace = True)
train_melt.drop("median_7", axis = 1, inplace = True)

train_melt.date_num = pd.to_numeric(train_melt.date_num)

In [80]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)

x_train = x_train.fillna(0)
x_valid = x_valid.fillna(0)

reg.fit(x_train, label_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [81]:
pred = reg.predict(x_valid)

In [82]:
y_true = np.array(label_valid)
y_pred = np.array(pred)

denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
diff = np.abs(y_true - y_pred) / denominator
diff[denominator == 0] = 0.0

print np.nanmean(diff)

129.897826373


In [53]:
x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
bst = run_xgb(x_train, label_train, x_valid, label_valid)

# bst = run_xgb(train_melt, label_train_melt)

[0]	train-rmse:113044	validation-rmse:121182	train-smape:176.038	validation-smape:174.376
Multiple eval metrics have been passed: 'validation-smape' will be used for early stopping.

Will train until validation-smape hasn't improved in 10 rounds.
[10]	train-rmse:112826	validation-rmse:121147	train-smape:162.418	validation-smape:163.085
[20]	train-rmse:112701	validation-rmse:121139	train-smape:161.8	validation-smape:162.643
Stopping. Best iteration:
[16]	train-rmse:112741	validation-rmse:121141	train-smape:161.605	validation-smape:162.391



In [41]:
importance = bst.get_fscore()

importance = sorted(importance.items(), key=operator.itemgetter(1))

df_imp = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_imp['fscore'] = df_imp['fscore']/df_imp['fscore'].sum()

print df_imp

     feature    fscore
0    mean_60  0.049815
1  median_60  0.065298
2    weekday  0.139010
3   date_num  0.217772
4    mean_30  0.235947
5  median_30  0.292158


In [16]:
def get_preds(test_df):
    
    d_test = xgb.DMatrix(test_df)
    p_test = bst.predict(d_test)
    
    return pd.Series(p_test)

In [17]:
def get_overall_df():
    
    key_wide = pd.DataFrame()
    key_wide["page"] = np.unique(key.trunc_page).tolist()
    for i in np.unique(key.date).tolist():
        key_wide[i] = 0

    overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
    overall_df.drop("page", axis = 1, inplace = True)
    
    return overall_df

In [18]:
def get_test_df(overall_df):
    
    test_df = pd.DataFrame()

    overall_df = pd.concat([overall_df.Page, overall_df.iloc[:, -120:]], axis = 1)
    pred_index = overall_df.columns.tolist().index('2017-01-01')
    
    for i in range(pred_index, len(overall_df.columns)):

        last_date = overall_df.columns.tolist()[i]
        print last_date
        last_index = i

        test_feats = pd.DataFrame()
        
        test_feats["Page"] = overall_df.Page
        test_feats["date"] = last_date
        
        test_feats["mean_7"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 7)
        test_feats["mean_30"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 30)
        test_feats["mean_60"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 60)

        test_feats["median_7"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 7)
        test_feats["median_30"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 30)
        test_feats["median_60"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 60)

#         test_feats["mean_diff_7"] = overall_df.apply(get_mean_diff_test, axis = 1, last_index = last_index)
#         test_feats["last_day_visits_None"] = overall_df.apply(get_last_day_visits_test, axis = 1, last_index = last_index)

#         test_feats["month"] = last_date.split("-")[1]
        test_feats["date_num"] = pd.to_numeric(last_date.split("-")[2])
        test_feats["weekday"] = datetime.datetime.strptime(last_date, '%Y-%m-%d').date().weekday()
                                
        test_df = pd.concat([test_df, test_feats])
        
    return test_df

In [24]:
overall_df = get_overall_df()
test_df = get_test_df(overall_df)
test_df.to_csv("test_feats.csv", index = False)

2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07
2017-01-08
2017-01-09
2017-01-10
2017-01-11
2017-01-12
2017-01-13
2017-01-14
2017-01-15
2017-01-16
2017-01-17
2017-01-18
2017-01-19
2017-01-20
2017-01-21
2017-01-22
2017-01-23
2017-01-24
2017-01-25
2017-01-26
2017-01-27
2017-01-28
2017-01-29
2017-01-30
2017-01-31
2017-02-01
2017-02-02
2017-02-03
2017-02-04
2017-02-05
2017-02-06
2017-02-07
2017-02-08
2017-02-09
2017-02-10
2017-02-11
2017-02-12
2017-02-13
2017-02-14
2017-02-15
2017-02-16
2017-02-17
2017-02-18
2017-02-19
2017-02-20
2017-02-21
2017-02-22
2017-02-23
2017-02-24
2017-02-25
2017-02-26
2017-02-27
2017-02-28
2017-03-01


In [44]:
# test_df = pd.read_csv("test_feats.csv")
test_df[test_df.Page == "2PM_zh.wikipedia.org_all-access_spider"]

,Page,date,mean_7,mean_30,mean_60,median_7,median_30,median_60,date_num,weekday,Visits
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-01,28.142857,25.000000,31.433333,26.0,22.0,18.5,1,6,24.135771
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-02,30.333333,25.068966,31.779661,28.0,22.0,19.0,2,0,24.135771
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-03,34.600000,25.285714,32.086207,30.0,22.0,19.0,3,1,24.135771
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-04,35.750000,25.444444,32.192982,35.5,22.0,19.0,4,2,24.135771
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-05,30.333333,24.615385,32.571429,26.0,22.0,19.0,5,3,24.135771
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-06,23.000000,24.480000,32.781818,23.0,22.0,19.0,6,4,24.135771
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-07,20.000000,24.583333,33.129630,20.0,21.0,19.0,7,5,24.135771
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-08,28.142857,24.695652,33.490566,26.0,20.0,19.0,8,6,24.135771
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-09,28.142857,22.863636,33.096154,26.0,19.5,19.0,9,0,24.135771
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-10,28.142857,22.666667,33.647059,26.0,19.0,19.0,10,1,24.135771


In [42]:
# test_df = pd.read_csv("test_feats.csv")
# prediction = get_preds(test_df[["mean_7","mean_30","mean_60","median_7","median_30","median_60","date_num", "weekday"]])
prediction = get_preds(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday"]])

In [43]:
# pred_index = overall_df.columns.tolist().index("2017-01-01")

# pred_df = overall_df.iloc[:, pred_index:]
# pred_df["Page"] = overall_df.Page

# pred_df_melt = pd.melt(pred_df, id_vars=['Page'], var_name="date", value_name="Visits")        

test_df["Visits"] = prediction
# pred_df = pd.concat([test_df[["Page", "date"]], prediction], axis = 1)
pred_df = test_df[["Page", "date", "Visits"]]

pred_id_visits = pd.merge(pred_df, key, left_on=["Page", "date"], right_on=["trunc_page", "date"], how="inner")[["Id", "Visits"]]

pred_id_visits.to_csv("pred_xgb_4.csv", index = False)

